In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


Nowy użytkownik w GA4 ma ga_session_number parametr zdarzeń w zestawie BigQuery = 1. Po powrocie użytkownik ma ten sam parametr zwiększony o 1.

Użytkownik jest zaangażowany gdy ma engagement_time_msec parametr zdarzeń większy niż 0 

Retencja użytkownika oznacza prawdopodobieństwo, że użytkownik powróci do witryny po jej pierwszej wizycie. Retencja jest zwykle mierzona w odstępach dni lub tygodni między tymi wizytami.

Count New And Returning Users

In [ ]:
%%bigquery --project

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_number') as session_number,
    max((select value.int_value from unnest(event_params) where key = 'engagement_time_msec')) as engagement_time_msec
from
    `ga.events_*`
where
    _table_suffix between '20210601' and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
group by
    user_pseudo_id,
    session_id,
    session_number)

select
    count(distinct case when session_number = 1 and engagement_time_msec > 0 then user_pseudo_id else null end) as new_users,
    count(distinct case when session_number > 1 and engagement_time_msec > 0 then user_pseudo_id else null end) as returning_users
from 
    prep

,new_users,returning_users
0,141067,56833


Calculate User Retention 1/2

In [ ]:
%%bigquery --project 

select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_number') as session_number,
    max((select value.int_value from unnest(event_params) where key = 'engagement_time_msec')) as engagement_time_msec,
    parse_date('%Y%m%d',event_date) as session_date,
    first_value(parse_date('%Y%m%d',event_date)) over (partition by user_pseudo_id order by event_date) as first_session_date
from
    `ga.events_*`
where
    _table_suffix between format_date('%Y%m%d',date_sub(current_date(), interval 43 day)) and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
group by
    user_pseudo_id,
    session_id,
    session_number,
    event_date
order by
    user_pseudo_id,
    session_id,
    session_number

LIMIT 4

,user_pseudo_id,session_id,session_number,engagement_time_msec,session_date,first_session_date
0,+059AC+LLVKWCU+QVhwCiHDmNhWdt1ETwvJTQS+NWt0=.1...,1627615503,1,18507.0,2021-07-30,2021-07-30
1,+0hQgt0DVhMmxT4ba6MesedCYNbMY4QEpqvcndlQKkA=.1...,1629912162,1,6.0,2021-08-25,2021-08-25
2,+0hQgt0DVhMmxT4ba6MesedCYNbMY4QEpqvcndlQKkA=.1...,1630482088,2,NaN,2021-09-01,2021-08-25
3,+1OyDZ8FfKesmc63OLA1qr8XBn+hfAE93NSMPxVq480=.1...,1629456719,1,2216.0,2021-08-20,2021-08-20


Calculate User Retention 2/2

In [ ]:
%%bigquery --project

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_number') as session_number,
    max((select value.int_value from unnest(event_params) where key = 'engagement_time_msec')) as engagement_time_msec,
    parse_date('%Y%m%d',event_date) as session_date,
    first_value(parse_date('%Y%m%d',event_date)) over (partition by user_pseudo_id order by event_date) as first_session_date
from
    `ga.events_*`
where
    _table_suffix between format_date('%Y%m%d',date_sub(current_date(), interval 43 day)) and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
group by
    user_pseudo_id,
    session_id,
    session_number,
    event_date
order by
    user_pseudo_id,
    session_id,
    session_number)

select 
    date_diff(session_date,first_session_date,day) as day,
    count(distinct case when session_number = 1 and engagement_time_msec > 0 then user_pseudo_id else null end) as new_users,
    count(distinct case when session_number > 1 and engagement_time_msec > 0 then user_pseudo_id else null end) as returning_users,
    count(distinct case when session_number > 1 and engagement_time_msec > 0 then user_pseudo_id else null end) / max(count(distinct case when session_number = 1 and engagement_time_msec > 0 then user_pseudo_id else null end)) over () as retention_percentage
from 
    prep
group by 
    day
order by 
    day
LIMIT 4

,day,new_users,returning_users,retention_percentage
0,0,61802,19289,0.312110
1,1,237,5463,0.088395
2,2,9,2638,0.042685
3,3,6,2368,0.038316


Calculate User Retention By Cohort

In [ ]:
%%bigquery --project 

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_number') as session_number,
    max((select value.int_value from unnest(event_params) where key = 'engagement_time_msec')) as engagement_time_msec,
    parse_date('%Y%m%d',event_date) as session_date,
    first_value(parse_date('%Y%m%d',event_date)) over (partition by user_pseudo_id order by event_date) as first_session_date
from
    `ga.events_*`
where
    _table_suffix between format_date('%Y%m%d',date_sub(current_date(), interval 100 day)) and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
group by
    user_pseudo_id,
    session_id,
    session_number,
    event_date
order by
    user_pseudo_id,
    session_id,
    session_number)

select
    distinct concat(extract(isoyear from first_session_date),'-',format('%02d',extract(isoweek from first_session_date))) as year_week,
    count(distinct case when date_diff(session_date,first_session_date,isoweek) = 0 and session_number >= 1 and engagement_time_msec > 0 then user_pseudo_id end) as week_0,
    count(distinct case when date_diff(session_date,first_session_date,isoweek) = 1 and session_number > 1 and engagement_time_msec > 0 then user_pseudo_id end) as week_1,
    count(distinct case when date_diff(session_date,first_session_date,isoweek) = 2 and session_number > 1 and engagement_time_msec > 0 then user_pseudo_id end) as week_2,
    count(distinct case when date_diff(session_date,first_session_date,isoweek) = 3 and session_number > 1 and engagement_time_msec > 0 then user_pseudo_id end) as week_3,
    count(distinct case when date_diff(session_date,first_session_date,isoweek) = 4 and session_number > 1 and engagement_time_msec > 0 then user_pseudo_id end) as week_4,
    count(distinct case when date_diff(session_date,first_session_date,isoweek) = 5 and session_number > 1 and engagement_time_msec > 0 then user_pseudo_id end) as week_5
from
    prep
group by 
    year_week 
order by
   year_week
LIMIT 4

,year_week,week_0,week_1,week_2,week_3,week_4,week_5
0,2021-21,9178,2162,1525,1327,1200,1006
1,2021-22,15170,2319,1631,1303,1153,919
2,2021-23,14150,1694,1077,881,719,610
3,2021-24,12701,1394,854,665,596,488
